# Notes


The leaf nodes are the character nodes

 Non-leaf nodes are the merge nodes

#### Padding

Inorder to determine which bit was the last bit, and how manuy extra bits(0's) have been placed at the last, we require Padding.

How much part we have paddeed, is present in the first 8 bits

In [7]:
import heapq, os

class BinaryTreeNode:
    def __init__(self, value, freq):
        self.value = value
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self,other): #Less than func

        return self.freq < other.freq

    def __eq__(self,other):     #Equal To Function
        return self.freq == other.freq

class HuffmanCoding:

    def __init__(self,path):
        self.path = path
        self.__heap = []
        self.__codes = {}
        self.__reverse_codes = {}
    
    def __make_frequency_dict(self,text):

        freq_dict={}

        for char in text:
            freq_dict[char] = freq_dict.get(char,0)+1
        
        return freq_dict

    def __buildHeap(self,freq_dict):
        
        for key in freq_dict:
            frequency = freq_dict[key]
            binary_tree_node = BinaryTreeNode(key,frequency)
            heapq.heappush(self.__heap, binary_tree_node)
        
    def __buildTree(self):
        while (len(self.__heap) > 1):
            binary_tree_node_1 = heapq.heappop(self.__heap)
            binary_tree_node_2 = heapq.heappop(self.__heap)
            freq_sum = binary_tree_node_1.freq + binary_tree_node_2.freq
            newNode = BinaryTreeNode(None, freq_sum)
            newNode.left = binary_tree_node_1
            newNode.right = binary_tree_node_2
            heapq.heappush(self.__heap, newNode)
        return
    
    def __buildCodesHelper(self,root,curr_bits):
        
        if root is None:
            return
        
        if root.value is not None:
            self.__codes[root.value] = curr_bits
            self.__reverse_codes[curr_bits] = root.value   # Map for storing reverse codes like 'a' -> 101
            return

        
        self.__buildCodesHelper(root.left,curr_bits+"0")
        self.__buildCodesHelper(root.right,curr_bits+"1")

    def __buildCodes(self):
        root = heapq.heappop(self.__heap)
        self.__buildCodesHelper(root, "")

    def __getEncodedText(self,text):
        encoded_text = ''
        for char in text:
            encoded_text+=self.__codes[char]

        return encoded_text

    def __getPaddedEncodedText(self,encoded_text):
        padded_amount = 8 - (len(encoded_text)%8)

        for i in range(padded_amount):
            encoded_text +='0'

        padded_info = "{0:08b}".format(padded_amount)   
        '''
        # First 0 suggests that the first argument in the format)_ func needs to be taken into account.
        #  After colon, 08b says that convert it  into binary of 8 bits. 
        
        '''
        
        padded_encoded_text = padded_info + encoded_text

        return padded_encoded_text


    def __getBytesArray(self,padded_encoded_text):
        array = []
        for i in range(0,len(padded_encoded_text),8):
            byte = padded_encoded_text[i:i+8]   #creating combination of 8 bits
            array.append(int(byte,2))   #the 2 after comma suggests the base of 2. So it will be converted to integer from binary in string

        return array


    def compress(self): 
        '''
        # Give a path to file, go to the file, read the text, compress, output to binary file
        #  Get file from path

        # read text path from file

        '''
        
        file_name, file_extension = os.path.splitext(self.path)   # Storing the file path and file extension from the text file using os module in python

        output_path = file_name + ".bin"    # Stoing the output file as '.bin

        with open(self.path,'r+') as file, open(output_path,'wb') as output:     
            '''
            #Opening the file as 'r+' = read and write mode and opening the ouput path as 'wb+' = write in binary mode

            # make freq dictionary using the text

            '''

            text = file.read()
            text = text.rstrip()  # Removes trailing spaces

            freq_dict = self.__make_frequency_dict(text)
            
            # Construct the heap from the frequency dictionary

            self.__buildHeap(freq_dict)


            #Construct the binary tree
            self.__buildTree()

            # Construct codes from binary tree

            self.__buildCodes()
            
            # Creating the encoded text using the texts

            encoded_text = self.__getEncodedText(text)

            #Put the encoded text into the binary file
    
            
            # Pad this encoded text

            padded_encoded_text = self.__getPaddedEncodedText(encoded_text)

            bytes_array = self.__getBytesArray(padded_encoded_text)

            # Return the binary file as output

            final_bytes = bytes(bytes_array) #  Python function, to convert bytes array into bytes
            output.write(final_bytes)   # Writing the output file
        print('Compressed')
        return output_path


    def __removePadding(self, text):
        padded_info = text[:8]
        extra_padding = int(padded_info,2)

        text = text[8:]
        text_after_padding_removed = text[:-1*extra_padding]  #if extra_padding is 4, it will slice till last 4 characters.
        return text_after_padding_removed    
    

    def __decodeText(self,text):

        decoded_text = ''
        current_bits = ''

        for bit in text:
            current_bits +=bit
            if current_bits in self.__reverse_codes:
                character = self.__reverse_codes[current_bits]
                decoded_text+=character
                current_bits = ''
        
        return decoded_text


    def decompress(self,input_path):
        file_name, file_extension = os.path.splitext(self.path)   # Storing the file path and file extension from the text file using os module in python

        output_path = file_name + "decompressed"+".txt"    # Stoing the output file as 'filename decompressed.txt'

        with open(input_path,'rb') as file, open(output_path,'w') as output:     #Opening the file as 'rb' = read binary and opening the ouput path as 'w' = write mode
            
            bitstring = ''  # inorder to read the binary string 
            
            byte = file.read(1)  #Reads each byte one by one 
            
            while(byte):   #Reads till byte is null
            
                byte = ord(byte)  # Gives the Ascii integer corresponding to a character
            
                bits = bin(byte)[2:].rjust(8,'0')  
                '''
                 #here bin gives the actual binary of the number found in byte. Now the bin stores as 0b______,
                 #so to omit 0b, we do [2:].... The rjust(8,'0') function is saying that if the string is not of len 8, append 0's to make it len of 8. 

                 '''
                bitstring += bits
                byte = file.read(1)

            ##Padding removal

            actual_text =  self.__removePadding(bitstring)
            decompressed_text = self.__decodeText(actual_text)
            output.write(decompressed_text)
            print('Decompressed')

path = 'C:/Users/abhra/OneDrive/Desktop/sample-2mb-text-file.txt'
h= HuffmanCoding(path)
output_path = h.compress() # for compression
h.decompress(output_path)     # For decompression

Compressed
Decompressed


In [4]:
path = 'C:/Users/abhra/OneDrive/Desktop/sample-2mb-text-file.txt'
path.

'C:/Users/abhra/OneDrive/Desktop/sample-2mb-text-file.txt'

In [4]:
print(ord('A'))

65


In [5]:
bin(65)

'0b1000001'